# Evolutionary algorithms analysis

Complete run of the evolutionary algorithms analysis. This notebook is used to run different evolutionary algorithms and test them on benchmark functions from benchamrk_function.py file. The results are saved in the results folder. 

Analysis is done via average ranks, with specific notes on outliers.

In [4]:
# Evo algorithms setup

# Imports
import time
import source.evolutionary_algorithms as ea
import numpy as np
import source.helpers as hp
import importlib
import inspect
import sys
import os

# Load benchmark functions
module_path = os.path.abspath(os.path.join('./source'))
sys.path.append(module_path)
benchmarks = importlib.import_module('benchmark_functions')
functions = [obj for _, obj in inspect.getmembers(benchmarks) if inspect.isfunction(obj)]

# Hyperparameters
DIMENSIONS = [2 ] #, 10, 30]

lower_bound = -100.0
upper_bound = 100.0

POPULATION_SIZES = [10, 20, 50]
FUNCTION_EVALUATIONS = 2000
RESULTS = dict()
REPEATS = 30

## Running ea

Run ea for benchmark functions and write results to global variables.


In [5]:
USE_PERSISTENT_DATA = True
FILENAME = f"results/partial_run_1702765089.636717.json" # Rename this in case of 

if USE_PERSISTENT_DATA:
    RESULTS = hp.load_results(FILENAME)
    print("Loaded results from file")
else:
    for i, dim in enumerate(DIMENSIONS):
        BOUNDS = np.array([[lower_bound, upper_bound]] * DIMENSIONS[i])
        pop_size = POPULATION_SIZES[i]
        evaluations = dim * FUNCTION_EVALUATIONS
        dimension_results = []
        print(f"Calculating dimension {dim} - ", end="")
        counter = 0

        for function in functions:
            runs = []
            counter += 1
            
            for _ in range(REPEATS):
                row = [0, 0, 0, 0, 0]
                row[0] = ea.differential_evolution(function, BOUNDS, pop_size, max_evaluations=evaluations, F=0.8, strategy="rand/1/bin")[1]
                row[1] = ea.differential_evolution(function, BOUNDS, pop_size, max_evaluations=evaluations, F=0.5, strategy="best/1/bin")[1]
                row[2] = ea.particle_swarm_optimization(function, BOUNDS, pop_size, max_evaluations=evaluations)[1]
                row[3] = ea.soma_ato(function, BOUNDS, pop_size, max_evaluations=evaluations)[1]
                row[4] = ea.soma_ata(function, BOUNDS, pop_size, max_evaluations=evaluations)[1]
                runs.append(row)

            print("#", end="")
            if counter % 5 == 0:
                print("|", end="")

            dimension_results.append(runs)
        
        RESULTS[dim] = dimension_results
        print("")
        
    hp.save_results(RESULTS, f"results/partial_run_{time.time()}.json")


Loaded results from file


## Analyze results

Legend/note:
- **Rank**: evolutionary algorithm's average rank for benchmark function across number of runs (REPEATS)
- **Rank Avg.**: distance between Rank and Total Avg. Rank
- **Total Avg. Rank**: average rank of evolutionary algorithm across all benchmark functions
- **Chi square**: used to test if Rank Avg. is significantly different from Average
  - if p-value is less than 0.05 then Rank Avg. is significantly different from Average

In [6]:
for dim in RESULTS:
    print(f"\nResults for {dim} dimensions:\n")
    ranking_table = []
    hp.table_header()
    result_dict = dict()
    for i, fun in enumerate(RESULTS[dim]):
        res = np.zeros(5)
        for run in fun:
            res += hp.rank_array(run)
        res /= REPEATS 
        result_dict[functions[i]._custom_name] = res
        ranking_table.append(res)

    average_ranks = np.mean(ranking_table, axis=0)
    avg_distance = 0
    
    for pair in result_dict:
        result_rank = result_dict[pair]
        distance = hp.euclidean_distance(result_rank, average_ranks)
        avg_distance += distance
        hp.table_row(pair, result_rank, distance)

    avg_distance /= len(result_dict)
    
    hp.table_footer(average_ranks, avg_distance)
    chi, p = hp.friedman_test(ranking_table)
    print("\nFriedman test:")
    print("Chi-square: {:>14.6}\nP-value: {:>18.6}".format(chi, p))

    print("\n\n")


Results for 2 dimensions:

----------------------------------------------------------------------------------------------------------------------------------------
|    Evo. Algorithm → |     DE Rand 1     |     DE Best 1     |        PSO        |  SOMA all-to-one  |  SOMA all-to-all  | Rank Avg.  |
| Benchmark ↓         |      (Rank)       |      (Rank)       |      (Rank)       |      (Rank)       |      (Rank)       |   (Diff)   |
----------------------------------------------------------------------------------------------------------------------------------------
| Ackley 1st          |       2.77        |       3.33        |       2.80        |       3.10        |       4.07        |    1.07    |
| Ackley Altered      |       3.07        |       4.03        |       1.37        |       2.30        |       4.23        |    1.89    |
| Alpine 1st function |       1.53        |       3.10        |       2.00        |       3.43        |       4.97        |    0.81    |
| Alpine 2nd 

### Explanation of outliers 
*(based on data from results/full_run.json)*

#### Dim 2

#### Dim 10

#### Dim 30